<a href="https://colab.research.google.com/github/andrewmlu/super-stable-rep/blob/main/CS242_Project_Scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
# from google.colab import drive
# import os
# drive.mount('/content/drive')
# os.chdir("drive/My Drive/!!!Drive/CS242-SD-imgs")

In [134]:
import requests
import json
from PIL import Image
import time
import re
import math
import numpy as np

In [114]:
url = "https://stablediffusionapi.com/api/v3/text2img"

def call_api(prompt, negative_prompt = None):

  # temp api key included
  payload = json.dumps({
    "key": "tgc1aN0Kk9ieQBgoVT1dssH0hvpeEap2YOMBEJIHYyUo2wnenimTpln5tnPO",
    "prompt": prompt,
    "negative_prompt": negative_prompt,
    "width": "512",
    "height": "512",
    "samples": "4",
    "num_inference_steps": "31",
    "seed": None,
    "guidance_scale": 7.5,
    "safety_checker": "yes",
    "multi_lingual": "no",
    "panorama": "no",
    "self_attention": "no",
    "upscale": "no",
    "embeddings_model": None,
    "webhook": None,
    "track_id": None
  })

  headers = {
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)

  return response


In [102]:
# response = call_api("golden labrador in a field of sunflowers")
# img_srcs = json.loads(response.text)['output']
# img_srcs

In [104]:
def save_img(url):

  # This statement requests the resource at
  # the given link, extracts its contents
  # and saves it in a variable
  data = requests.get(url).content
  # print(data)

  # Opening a new file named img with extension .png
  # This file would store the data of the image file
  filename = extracted_part = re.search(r'[^/]*$', url).group()
  f = open(filename,'wb')

  # Storing the image data inside the data variable to the file
  f.write(data)
  f.close()

  # Opening the saved image and displaying it
  # img = Image.open(filename)
  # img.show()


In [111]:
def save_imgs_from_prompt(prompt, negative_prompt=None, num_imgs=12):
  response = call_api(prompt, negative_prompt)
  img_srcs = json.loads(response.text)['output']
  for i in range(math.ceil(num_imgs/4)):
    for src in img_srcs:
      save_img(src)

In [112]:
save_imgs_from_prompt("golden labrador in a field of sunflowers")

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x00\x00\x00\x02\x00\x08\x02\x00\x00\x00{\x1aC\xad\x00\x01\x00\x00IDATx\x9cl\xfd\xdd\x9a\xe4H\x92,\x88\x89\xa8\x1a\xe0\x11Y==\xfbG^\xf1\xfd\x9f\x83\xcf\xc1\x0b^p\xf7\xfb\xb8<g\xa6\xbb*3\xc3\x1d\x80\xa9\xaa\xf0B\r\x9e\xd9s6\xbb\xba*\xd2\xc3\x1d\x0e\x18`\xfa#**\xca\xff\xe7\xff\xe7Y\xb5\x15\xcb\n4\x16\x05\x88"\tB\x00\x01\x02"\x00\x02\x00\x04\xf6\x0f\xe0\xfb\x15P\xeb\xb5\xf7\xcb"\xa0\xf7\xdf\xd9?\xf4+\xba\x0fu\xbfD\xfc\xfa\xf4z\xf5>4\x01\x08\xf7\xd1\xc9\x7f=\x13\x00\xf8\xfd\x8b!\xbcO\xf7\xd7\x97\xff\xd7C\xbf\xbf\xe2\xfe\xcb\xafc\xfe\xeb\x9b\xefc\xf0\xd79\x13\x80\xd6\xcbz_\xda\xafW\xf4\xfbq\xef\xf7\x83\x10H\xa9\xdf\xaf\x7f\xfd\xa5\xec\xd7\xd5\x01\x10\x85\xf7*\xbf\xdf\xaa\xff\xe1\xdc~\xad\xe7\x7f\xbd\xb6_?\xae#\xdew\x91\x00\t\xde\x7f\xe7\xbdT\xa4\xdegM\xae\xf3\x05\x05\xc8~\xbfR\xad+Y\x9fZ\'\xc6\xb5\x06X?\xf1\xb7S\xe0}\xeb\x04I\xeb8\xf6\xeb9\x80\x13\x02S0\xae\xd3\xd3\xbd\x8e\xbfn\nQ\xf7SBBZ_\xd1\xebVZ\x0b! \xeeU\xa9\xfb\x14\xfc\xf7\x9b\xd8o\x13

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x00\x00\x00\x02\x00\x08\x02\x00\x00\x00{\x1aC\xad\x00\x01\x00\x00IDATx\x9c\\\xfd\xdbr\xec\xc8\x924\x8cyDd\x02U$\xd7\xea\xee=3\x9fL\xd2\x85.\xa4\x1b\x99\x9eC\xa6\xf7\x7f\x03I\xbf}\xd27\x87>\xacEV\x01\x99qp]$\xc8\xde\xa3\xdef\xbb\x9bd\x1d\x80L \xc2\xc3\xdd# \xff\xf3}"5QZ(\x82\x02\n\x85"\x02\x01\x08\x10\x02@\xc0\xcf\x1f\xa1\xc0\xf5\xb7\xeb\x1f!\x08\xca\xd7\xaf\xe5\xebo\x02\xf9|\x11\xff\xdb\x9f\xe4\xf3\xafr\xfd\x92\x02\x80\x9f\xaf\xffo\x1f\xbf\xdeN\x12\xf2\xf9\x01\xd7\x97}}\xa6\\\xaf\x01\x00\x91\xcfw\x13\\\xc7\x06*\x00\x91\xfa\xfcp\x15\x01\x0b\\o\xf9\xfb`\x8a(\xf0\xfa\x1aB\x00\x12\x14\x11Q\xa2\x84\x04\t\x11B\xd6\xd7\x00(\xae\xd7\xcb\xf5\x1e\x82\x80\x88\x08y\x1d+\xa4\x04"\xd0\xeb\xc7\xcf\xaf\x14\x11\x10$!\xa2BR\xd6\x02\xf0\xfa\xf3\xd7\xba\xc9\xfat\xac\xe3\xa7\\\xe7x}!\x00\xca\xe7\xb2\x89\x10%\x7f\xaf\xc5ZY\x05D\x00\xe1\xe7\x82\xc9\xd7\xc7\x7fn\x08\x01\x11\nI\xea:f\xca\xe7\xa6\xb3\xd6\xd7}\xae2\xd7Q\x7f\x1e\xdf\xfa\x1b\xaf\xa5\xe4\xe7\xe2_\x8

In [116]:
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer

In [117]:
from transformers import CLIPProcessor, CLIPModel

# Load the pre-trained CLIP model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Text string
text = "a banana sitting at the beach"

# Prepare the text
inputs = processor(text=text, return_tensors="pt")

# Perform the encoding
text_features = model.get_text_features(**inputs)

# The text_features tensor is the encoded representation of your text
print(text_features)

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tensor([[-6.9399e-02,  4.6953e-01, -2.2282e-01, -1.8533e-01,  1.8412e-01,
         -5.3928e-02,  5.8774e-02, -9.3234e-01, -2.3446e-01, -7.7533e-02,
         -1.7197e-01, -3.5734e-01,  4.0063e-01,  1.7931e-01,  2.7660e-01,
          1.0494e-01,  1.3747e-01, -2.2337e-01, -2.1525e-01, -3.8099e-01,
          4.9047e-01, -4.6832e-01,  1.2928e-01, -3.0012e-01, -8.2612e-02,
         -6.0348e-02, -1.2062e-02, -1.4567e-01, -1.9723e-01, -6.1405e-02,
         -1.5761e-01, -2.0645e-01, -1.9105e-01,  1.3132e-01, -3.7881e-02,
         -8.0036e-02,  4.9568e-02,  1.4898e-01,  3.0919e-01, -2.2620e-01,
         -3.0060e-01, -2.5104e-01, -7.2611e-02,  3.2783e-01, -8.4561e-02,
          3.2503e-01,  2.9675e-01, -1.8957e-01,  1.1224e-01,  6.3529e-02,
         -4.9092e-02,  1.3483e-01, -5.9284e-02, -1.0503e-02, -1.9684e-01,
         -3.3757e-01, -1.2274e-01, -9.0355e-02, -3.0661e-01, -8.8419e-02,
          1.6018e-01,  1.7066e-01, -1.0104e-01, -2.2407e-01,  1.1011e-01,
         -1.8230e-01, -1.3388e-01,  1.

In [122]:
# Text string
text = ["a banana sitting at the beach", "two bananas sitting at the beach", "a dog sitting at the becah"]

# Prepare the text
inputs = processor(text=text, return_tensors="pt", padding=True)

# Perform the encoding
text_features = model.get_text_features(**inputs)

# The text_features tensor is the encoded representation of your text
print(text_features)

tensor([[ 0.1699,  0.3451, -0.0828,  ...,  0.2795, -0.0243, -0.0554],
        [ 0.2219,  0.2785, -0.1488,  ...,  0.2776, -0.0453, -0.1471],
        [-0.0926, -0.1647, -0.3049,  ..., -0.2542, -0.1363,  0.1552]],
       grad_fn=<MmBackward0>)


In [123]:
text_features[0] @ text_features[1]

tensor(56.8696, grad_fn=<DotBackward0>)

In [124]:
text_features[0] @ text_features[2]

tensor(49.9023, grad_fn=<DotBackward0>)

In [125]:
text_features[1] @ text_features[2]

tensor(49.9115, grad_fn=<DotBackward0>)

In [136]:
from sklearn.metrics.pairwise import cosine_similarity

# Assuming text_features is a tensor with encoded text features
# Convert to numpy array if it's a PyTorch tensor
text_features_np = text_features.detach().numpy()

# Calculate cosine similarity
# This will return a matrix of similarities between each pair of texts
similarity_matrix = cosine_similarity(text_features_np)
np.fill_diagonal(similarity_matrix, np.NINF)

print(similarity_matrix[0])
print(np.argmax(similarity_matrix[0]))


[      -inf 0.9086889  0.66604084]
1


In [ ]:
strings = [f"a {1} next to a tree",
           f"a {1} on the left of a building",
           f"a {1} on the right of a wall",
           f"a {1} in a bowl",
           f"a {1} on top of a hill",
           f"a {1} held in a hand"]